## Gradient Boosting


In [ ]:
# Read CSV

deaths_simple = pd.read_csv('/Users/HudsonCavanagh/Dropbox/Capstone/cdc_data/deaths_simple.csv', delimiter='\t', encoding='utf-8')

#Quick Explore

pov_county_year_03_14['year'].value_counts()
pov_county_year_03_14.describe()
deaths_pop.isnull().sum()



#Concat

pop_03_14 = pd.concat([pop_03, pop_04, pop_05, pop_06, pop_07, pop_08, pop_09, pop_10_14], axis=0) #this works 1) need to make floats 2) need more data



#Merge

deaths_pop = pd.merge(deaths_pop, pop_03_14, how='left', left_on=['year', 'county_code'], right_on=['year', 'county_code'])
deaths_pop.reset_index(inplace=1)


#Groupby

deaths_pop = cdc_03_14.groupby(by=['year', 'county_code']).sum()



#Pivot Table
state_change = pd.pivot_table(deaths_pop, values=['drug_death_rate_100k'], index=['state', 'year'], aggfunc='mean')

#Dropping Values

#Rows
deaths_pop.drop(886, inplace=1)

deaths_pop.dropna(axis=0, subset=['working_pop', 'employed_pop', 'deaths_raw', 'population_est'], inplace=1)


#Cleaner Functions

def yearify(year):
    year = float(year)
    try:
        return int(year)
    except:  
        return year
    
def interator(example):
    try:
        example = str(example).split('.')[0]
        return int(example)
    except:
        return example
    
def death_clean(death):
    try:
        return float(death)
    except:
        return np.nan



#Feature Scaling

X = StandardScaler().fit_transform(features)
X = pd.DataFrame(X, columns=features.columns)

#PCA for later plots & model testing

pca_2 = PCA(n_components=2)
pca_3 = PCA(n_components=3)
pca_4 = PCA(n_components=4)
xPC_2 = pca_2.fit_transform(X)
xPC_3 = pca_3.fit_transform(X)
xPC_4 = pca_4.fit_transform(X)



#Train test split & Cross Validation

from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, BaggingRegressor
import seaborn as sns

rf = RandomForestRegressor()
rf_cv = RandomForestRegressor()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=50)
rf.fit(X_train, y_train)
ypred_rf = rf.predict(X_test)
s_rf_cv = cross_val_score(rf_cv, X, y, cv=stratk, n_jobs=-1)
    
s_rf_cv.mean()



#Basic GB Regression Implementation

gb_delta_14_mod = GradientBoostingRegressor(n_estimators=10000, learning_rate=.02)


gb_delta_14_mod.fit(X_03_13_mod, y_delta_grow_03_13) # works
gb_delta_14_ypred_mod = gb_delta_14.predict(X_14_mod)

#XG Boost

import xgboost

xg_class_model_cv = xgboost.XGBClassifier()

s_xg_cv = cross_val_score(xg_class_model_cv, X, y_bin, cv=stratk, n_jobs=-1)

s_xg_cv.mean()


# fit model no training data
xg_model = xgboost.XGBRegressor()
xg_model_cv = xgboost.XGBRegressor()
xg_model.fit(X_train, y_train)

s_xg_cv = cross_val_score(xg_model_cv, X, y, cv=stratk, n_jobs=-1)

# make predictions for test data
y_pred_xg = xg_model.predict(X_test)
s_xg_cv.mean()




#Basic GB Classification Implementation

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


y_bin_03_13 = y_03_13.apply(lambda x: 1 if x > median_drug_rate else 0)
y_bin_14_obs = y_14['drug_death_rate_100k'].apply(lambda x: 1 if x > median_drug_rate else 0)


xg_class_model_14 = xgboost.XGBClassifier()
xg_class_model_14.fit(X_03_13, y_bin_03_13)
y_class_14 = xg_class_model_14.predict(X_14)

xg_14_cm = confusion_matrix(y_bin_14_obs, y_class_14, labels=xg_class_model_14.classes_)



#Eval Metrics - 


#Regression

r2_score(y_delta_grow_14, gb_delta_14_ypred_mod)


sns.regplot(ypred_gbtree, y_test, color="orange")
plt.xlabel('Predicted Mortality Rate (Gradient Boosted)')
plt.ylabel('Actual Mortality Rate')
plt.show()


#Classification

import matplotlib.patheffects as path_effects

def plot_confusion_matrix(cm, title='Confusion matrix', labels=['Positive','Negative'], cmap=plt.cm.Blues):
   
   plt.figure(figsize=(7,7))
   plt.imshow(cm, interpolation='nearest', cmap=cmap)
   
   tick_marks = np.arange(2)
   plt.xticks(tick_marks, labels)
   plt.yticks(tick_marks, labels)
       
   plt.title(title)
   plt.ylabel('True label')
   plt.xlabel('Predicted label')
   plt.colorbar()
   plt.tight_layout()
   
   width, height = cm.shape
   
   for x in xrange(width):
       for y in xrange(height):
           plt.annotate(str(cm[x][y]), xy=(y, x), 
                       horizontalalignment='center',
                       verticalalignment='center',
                       color = 'white',
                       fontsize=18).set_path_effects([path_effects.Stroke(linewidth=1, foreground='black'),
                                                      path_effects.Normal()]) #The last line here adds a text outline
        
plot_confusion_matrix(xg_14_cm)







#Gradient Boost with Grid Search

this code work, yielded:
{'max_depth': 2,
 'max_features': 'auto',
 'min_samples_leaf': 5,
 'min_samples_split': 5}    


max_feature_list = ['auto']
param_test = {'max_depth':[2,5], 'max_features':max_feature_list, 'min_samples_split':[2,5], 'min_samples_leaf':[2,5]}

gb_delta_14_reg = GradientBoostingRegressor(n_estimators=100, learning_rate=.1)
gb_delta_14_class = GradientBoostingClassifier(n_estimators=10000, learning_rate=.02)

gs_delta_14_reg = GridSearchCV(gb_delta_14_reg, param_test, verbose=False, cv=15)
gs_delta_14_reg.fit(X_03_13_mod, y_delta_grow_03_13)
y_delta_14_gs_preds = gs_delta_14_reg.predict(X_14_mod)


gs_delta_14_reg.best_params_


#Feature Importance

rel_feature_import = sorted(zip(rf.feature_importances_, features.columns), reverse=True)
rel_feature_import = pd.DataFrame(rel_feature_import)
rel_feature_import


#Unsupervised Clustering

from sklearn.cluster import DBSCAN, KMeans
from sklearn.metrics import silhouette_score


db1 = DBSCAN(eps = 4, min_samples = 15).fit(X)  
db1labels = db1.labels_  
core_samples = db1.core_sample_indices_

print("Silhouette Coefficient: %0.3f" 
      % silhouette_score(X, db1labels), len(set(db1labels)), db1labels.mean())



#PCA 2-D Graphing


plt.style.use('fivethirtyeight')
xPC_2 = pd.DataFrame(xPC_2, columns=['pc1', 'pc2'])
xPC_2.head()

graph = xPC_2.plot(kind='scatter', x='pc1', y='pc2', figsize=(16,8))

for i, county in enumerate(deaths_pop['county_name']):
    graph.annotate(county, (xPC_2.iloc[i,0], xPC_2.iloc[i,1]))
    
    
#t-SNE 2-D Graphing

import datetime
from tsne import bh_sne
# tsne = manifold.TSNE(n_components=2, init='pca', random_state=0)
# t0 = datetime.time()
# X_tsne = tsne.fit_transform(features)


X_2d = bh_sne(X)
plt.style.use('fivethirtyeight')
plt.figure(figsize=(30, 10))
plt.scatter(X_2d[:,0],X_2d[:,1])
plt.title('t-SNE')
plt.xlabel('')
plt.ylabel('')


plt.show()